# Setting up conection 

In [ ]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import mapper, sessionmaker
from sqlalchemy import Column, Integer
from sqlalchemy.ext.declarative import declarative_base
from scrapy.selector import Selector
from sqlalchemy.sql import text as sa_text
import re

engine = create_engine('mysql+pymysql://root:@localhost/oerintegrationdb', echo=False)
Base = declarative_base(engine)

class Triple(Base):
    __tablename__ = 'triple'
    __table_args__ = {'autoload': True}


class CleanTriple(Base):
    __tablename__ = 'cleantriple'
    __table_args__ = {'autoload': True}

metadata = Base.metadata
Session = sessionmaker(bind=engine)
session = Session()

print("Ready DB")

In [ ]:
triples = session.query(Triple).filter(Triple.subject =="oasis")
print("Cleaned")

In [ ]:
source_raw = "OAsis"
for triple in triples:
    body = triple.object
    source = triple.source
    # Setting name
    name = Selector(text=body).xpath("//h2/text()").get()  
    name_triple =  name.replace(" ", "")
    name_triple =  name_triple.replace("/", "")
    name_triple =  name_triple.replace(":", "")
    print('_ _ _ _')
    print('name')
    print('_ _ _ _')
    print(name_triple)
    session.add(CleanTriple(subject_uri=source ,subject_id =source_raw, predicate='Name', object=name_triple))
    
    table_predicate = Selector(text=body).xpath("//td[@class='label-cell']/text()").getall()
    table_object = Selector(text=body).xpath("//td[@class='word-break']/text()").getall()
    if table_object is not None and table_predicate is not None:
        for predicate, object in zip(table_predicate, table_object):
            print(predicate, ' : ', object)
            session.add(CleanTriple(subject_uri=source ,subject_id =source_raw, predicate=predicate, object=object))
    
    files = Selector(text=body).xpath('//div[@class="file-wrapper row"]').getall()
    j=1
    obj_files = []
    if files is not None:
        for f in files:
            obj = source + '_file' + str(j)
            obj_files.append(obj)
            j+=1
            print('file : ', obj)
            session.add(CleanTriple(subject_uri=source ,subject_id =source_raw, predicate='file', object=obj))
            
        for file, o in zip(files, obj_files):
            img = Selector(text=file).xpath('//div[@class="file-wrapper row"]/div/div/a/img/@src').getall()
            fl = Selector(text=file).xpath('//a/@href').getall()
            img = ["%s%s" % ('http://oasis.col.org', s) for s in img]
            fl = ["%s%s" % ('http://oasis.col.org', s) for s in fl]
            files_predicate = Selector(text=file).xpath('//dl[@class="file-metadata dl-horizontal"]/dt/text()').getall()
            files_object = Selector(text=file).xpath('//dl[@class="file-metadata dl-horizontal"]/dd/text()').getall()
            if files_object is not None and files_predicate is not None:
                for predicate, object in zip(files_predicate, files_object):
                    print(predicate, ' : ', object)
                    session.add(CleanTriple(subject_uri=o ,subject_id =source_raw, predicate=predicate, object=object))
            print('img :', img[0])
            print('file : ', fl[0])
            session.add(CleanTriple(subject_uri=o ,subject_id =source_raw, predicate='image', object=img[0]))
            session.add(CleanTriple(subject_uri=o,subject_id =source_raw, predicate='url_file', object=fl[0]))
            
print("Done")     
session.commit()
session.close_all()